In [3]:
##processing for the thingy uhhh semg data parsing yes yes agreed##

import csv
import pandas as pd

def find_transition_times(filename):
    with open(filename, 'r') as file:
        reader = csv.DictReader(file)
        prev_label = None
        transition_times = [None, None]
        for row in reader:
            current_label = int(row['label'])
            if prev_label is not None:
                if prev_label == 0 and current_label == 1:
                    transition_times[0] = row['time']
                elif prev_label == 1 and current_label == 2:
                    transition_times[1] = row['time']
                # Break early if both transitions are found
                if transition_times[0] is not None and transition_times[1] is not None:
                    break
            prev_label = current_label
        return transition_times
    
def parse_emg_data(file_in, file_out, list):
  df = pd.read_csv(file_in)
  cols_to_drop = []
  index = 0
  for name in df.columns:
    if index == 2 or index == 4 or index == 6:
      cols_to_drop.append(name)
    index = index + 1
  
  df = df.drop(columns=cols_to_drop, axis=1)
  t1 = float(list[0])
  t2 = float(list[1])

  def label_fn(val: float) -> int:
    if val < t1:
      return 0
    elif val < t2:
      return 1
    else:
      return 2
  
  df['Label'] = df.iloc[:, 0].apply(label_fn)

  return df.to_csv(file_out)
    

In [ ]:
##run once dude trust me ts is gonna take a while.##
##all ts do is process all the semg data minimum 7 minutes##

directory = "parsed_emg"
subject_numbers = range(1,14)
trials = range(1,13)

for subject_number in subject_numbers:
    for trial in trials:
        self_perceived_path = "self_perceived_fatigue_index/" + "subject_" + str(subject_number) + "/Trial_" + str(trial) + ".csv"
        print("parsing self perceived path: " + self_perceived_path)
        labels = find_transition_times(self_perceived_path)
        semg_path = "sEMG_data/subject_" + str(subject_number) + "/trial_" + str(trial) + ".csv"
        if (trial % 2 == 1):
            semg_output = "parsed_emg/subject_" + str(subject_number) + "/trial_" + str(trial) + "(right)" + ".csv"
        else:
            semg_output = "parsed_emg/subject_" + str(subject_number) + "/trial_" + str(trial) + "(left)" + ".csv"
        if (subject_number == 13 and trial == 12):
            labels = [168.41, 400]
        print("parsing semg data: " + semg_path)
        parse_emg_data(semg_path, semg_output, labels)

        


In [7]:
df = pd.read_csv("parsed_emg/subject_1/trial_1(right).csv")
print(df)

        Unnamed: 0      X [s]  R BICEPS BRACHII: EMG 1 [V]  \
0                0   0.000000                    -0.000004   
1                1   0.000794                    -0.000007   
2                2   0.001588                    -0.000009   
3                3   0.002382                    -0.000016   
4                4   0.003176                    -0.000018   
...            ...        ...                          ...   
120445      120445  95.647444                     0.000003   
120446      120446  95.648238                     0.000007   
120447      120447  95.649032                     0.000007   
120448      120448  95.649826                     0.000012   
120449      120449  95.650620                     0.000014   

        R DELTOID ANTERIOR: EMG 2 [V]  R DELTOID MEDIUS: EMG 6 [V]  \
0                           -0.000022                 2.182040e-06   
1                           -0.000021                -1.787100e-06   
2                           -0.000022        